<a href="https://colab.research.google.com/github/nntadotzip/XLNet-sandbox/blob/main/XLNetQA_huggingfacePipeline_BertPretrainedTokenizerFast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import and Install

In [1]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
! pip install datasets transformers
! pip install pytorch-transformers

In [4]:
!huggingface-cli login


        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/token.
        (Deprecated, will be removed in v0.3.0) To login with username and password instead, interrupt with Ctrl+C.
        
Token: 
Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine

In [5]:
import torch
import transformers
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

from transformers import XLNetModel, XLNetTokenizer, XLNetForQuestionAnswering
from transformers import AdamW

from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla T4'

In [7]:
!git config --global credential.helper store

In [8]:
from huggingface_hub import notebook_login
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


In [9]:
!apt install git-lfs
!git config --global user.email "nntanh@hcmiu.edu.vn"
!git config --global user.name "nntadotzip"

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 2,129 kB of archives.
After this operation, 7,662 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 git-lfs amd64 2.3.4-1 [2,129 kB]
Fetched 2,129 kB in 0s (17.5 MB/s)
Selecting previously unselected package git-lfs.
(Reading database ... 155229 files and directories currently installed.)
Preparing to unpack .../git-lfs_2.3.4-1_amd64.deb ...
Unpacking git-lfs (2.3.4-1) ...
Setting up git-lfs (2.3.4-1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [10]:
from datasets import load_dataset
dataset = load_dataset("nntadotzip/iuQAchatbot", use_auth_token=True, field='data')

Using custom data configuration nntadotzip--iuQAchatbot-da3c79a8c8a167fc


  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/2.33M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/263k [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/nntadotzip--iuQAchatbot-da3c79a8c8a167fc/0.0.0/c90812beea906fcffe0d5e3bb9eba909a80a998b5f88e9f8acbd320aa91acfde. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['answers', 'context', 'id', 'question', 'title'],
        num_rows: 166
    })
    test: Dataset({
        features: ['answers', 'context', 'id', 'question', 'title'],
        num_rows: 19
    })
})

In [12]:
dataset["train"][0]

{'answers': {'answer_start': [4163],
  'text': ['Phòng Công tác Sinh viên: Được thành lập năm vào tháng 12/2009, Phòng Công tác Sinh viên (PCTSV) là đơn vị tham mưu giúp việc cho Ban Giám hiệu trong công tác thực hiện chế độ chính sách, học bổng Doanh nghiệp, khen thưởng-kỷ luật sinh viên']},
 'context': 'Phòng Đào tạo Sau đại học: Tham mưu cho Hiệu trưởng về các công tác quản lý đào tạo và bồi dưỡng sau đại học thuộc trách nhiệm và thẩm quyền. Liên hệ: Room O2 – 609Telephone: (84)-28-37244270. Ext: 3120Fax: (84-28) 37244271. Phòng Vật tư Thiết bị: Phòng Vật tư Thiết bị Liên hệ: Phòng O1 – 612, Trường Đại học Quốc tếKhu phố 6, Phường Linh Trung, Quận Thủ Đức, TP. HCMĐiện thoại: (028) 37244270 – số máy lẻ: 3555, 3246 Fax: (028) 37244271. Trung tâm Đổi mới sáng tạo và Chuyển giao công nghệ: Trung tâm là đơn vị sự nghiệp khoa học trực thuộc trường Đại Học Quốc Tế – Đại Học Quốc Gia TP. HCM, có chức năng cung cấp dịch vụ – nghiên cứu – đào tạo – tư vấn và chuyển giao công nghệ cho các doan

Review the Dataset

In [13]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [14]:
show_random_elements(dataset["train"])

# Preprocess the training data

In [15]:
model_checkpoint = "xlnet-base-cased"
batch_size = 8

In [31]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

In [32]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [33]:
tokenizer("Phòng Bảo đảm Chất lượng Giáo dục và Khảo thí:", " Tư vấn, tham mưu cho Ban Giám hiệu về chiến lược, các kế hoạch cải tiến hoạt động bảo đảm chất lượng")

{'input_ids': [101, 7642, 20142, 2118, 139, 28641, 1186, 288, 28641, 1306, 20394, 28642, 1204, 181, 28265, 28660, 2118, 144, 1182, 5589, 1186, 173, 28661, 1665, 191, 9183, 148, 1324, 28641, 1186, 24438, 6212, 131, 102, 157, 28265, 191, 28642, 1179, 117, 24438, 2312, 182, 28265, 1358, 22572, 1186, 18393, 144, 1182, 5589, 1306, 20844, 28651, 1358, 191, 28648, 22572, 1182, 28647, 1179, 181, 28265, 28660, 1665, 117, 172, 5589, 1665, 180, 28647, 16358, 28640, 1732, 172, 28641, 1182, 189, 1182, 28647, 1179, 16358, 28640, 1204, 288, 28657, 2118, 171, 28641, 1186, 288, 28641, 1306, 22572, 28642, 1204, 181, 28265, 28660, 2118, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1,

In [34]:
max_length = 512 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

Let's find one long example in our dataset:

In [35]:
for i, example in enumerate(dataset["train"]):
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > 512:
        break
example = dataset["train"][i]

Token indices sequence length is longer than the specified maximum sequence length for this model (5018 > 512). Running this sequence through the model will result in indexing errors


Without any truncation, we get the following length for the input IDs:

In [36]:
len(tokenizer(example["question"], example["context"])["input_ids"])

5018

Now, if we just truncate, we will lose information (and possibly the answer to our question):

In [37]:
len(tokenizer(example["question"], example["context"], max_length=max_length, truncation="only_second")["input_ids"])

512

Note that we never want to truncate the question, only the context, else the only_second truncation picked. Now, our tokenizer can automatically return us a list of features capped by a certain maximum length, with the overlap we talked above, we just have to tell it with return_overflowing_tokens=True and by passing the stride:

In [38]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride
)

Now we don't have one list of input_ids, but several:

In [39]:
for x in tokenized_example["input_ids"][:2]:
    print(tokenizer.decode(x))

[CLS] Cho tôi [UNK] về Phòng Công tác Sinh viên [SEP] Phòng Đào tạo Sau đại học : Tham mưu cho Hiệu [UNK] về các công tác quản lý đào tạo và bồi [UNK] sau đại học thuộc trách nhiệm và thẩm quyền. Liên hệ : Room O2 – 609Telephone : ( 84 ) - 28 - 37244270. Ext : 3120Fax : ( 84 - 28 ) 37244271. Phòng Vật tư Thiết bị : Phòng Vật tư Thiết bị Liên hệ : Phòng O1 – 612, Trường Đại học Quốc tếKhu phố 6, Phường Linh Trung, Quận Thủ Đức, TP. HCMĐiện thoại : ( 028 ) 37244270 – số máy [UNK] : 3555, 3246 Fax : ( 028 ) 37244271. Trung tâm Đổi mới sáng tạo và Chuyển giao công nghệ : Trung tâm là đơn vị sự nghiệp khoa học trực thuộc trường Đại Học Quốc Tế – Đại Học Quốc Gia TP. HCM, có chức năng cung cấp dịch vụ – nghiên cứu – đào tạo – tư vấn và chuyển giao công nghệ cho các doanh nghiệp và các cơ quan chức năng Nhà nước. Trung tâm là cầu nối giữa giảng viên và [SEP]
[CLS] Cho tôi [UNK] về Phòng Công tác Sinh viên [SEP] Quốc Gia TP. HCM, có chức năng cung cấp dịch vụ – nghiên cứu – đào tạo – tư vấn và

Now this will give us some work to properly treat the answers: we need to find in which of those features the answer actually is, and where exactly in that feature. The models we will use require the start and end positions of these answers in the tokens, so we will also need to to map parts of the original context to some tokens. Thankfully, the tokenizer we're using can help us with that by returning an offset_mapping:

In [40]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride
)
print(tokenized_example["offset_mapping"][0][:100])

[(0, 0), (0, 3), (4, 5), (5, 6), (6, 7), (8, 11), (12, 13), (13, 14), (15, 17), (17, 18), (18, 20), (21, 22), (22, 23), (23, 25), (26, 27), (27, 28), (28, 29), (30, 33), (33, 34), (35, 36), (36, 37), (37, 38), (38, 39), (0, 0), (0, 2), (2, 3), (3, 5), (6, 7), (7, 8), (8, 9), (10, 11), (11, 12), (12, 13), (14, 16), (16, 17), (18, 19), (19, 20), (20, 21), (22, 23), (23, 24), (24, 25), (25, 26), (27, 28), (28, 31), (32, 33), (33, 34), (34, 35), (36, 38), (38, 39), (40, 42), (42, 43), (43, 44), (45, 51), (52, 53), (53, 54), (55, 56), (56, 57), (57, 58), (59, 60), (60, 61), (61, 63), (64, 65), (65, 66), (66, 67), (68, 69), (69, 70), (70, 71), (71, 72), (73, 74), (74, 75), (76, 77), (77, 78), (78, 79), (80, 81), (81, 82), (82, 83), (84, 85), (85, 86), (87, 88), (88, 89), (89, 90), (91, 96), (97, 99), (99, 100), (101, 102), (102, 103), (103, 104), (105, 106), (106, 107), (107, 108), (109, 111), (111, 112), (112, 113), (113, 114), (115, 116), (116, 117), (117, 118), (118, 120), (121, 122), (12

This gives, for each index of our input IDS, the corresponding start and end character in the original text that gave our token. The very first token ([CLS]) has (0, 0) because it doesn't correspond to any part of the question/answer, then the second token is the same as the characters 0 to 3 of the question:

In [41]:
first_token_id = tokenized_example["input_ids"][0][1]
offsets = tokenized_example["offset_mapping"][0][1]
print(tokenizer.convert_ids_to_tokens([first_token_id])[0], example["question"][offsets[0]:offsets[1]])

Cho Cho


So we can use this mapping to find the position of the start and end tokens of our answer in a given feature. We just have to distinguish which parts of the offsets correspond to the question and which part correspond to the context, this is where the sequence_ids method of our tokenized_example can be useful:

In [42]:
sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

It returns None for the special tokens, then 0 or 1 depending on whether the corresponding token comes from the first sentence past (the question) or the second (the context). Now with all of this, we can find the first and last token of the answer in one of our input feature (or if the answer is not in this feature):

In [43]:
answers = example["answers"]
start_char = answers["answer_start"][0]
end_char = start_char + len(answers["text"][0])

# Start token index of the current span in the text.
token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1

# End token index of the current span in the text.
token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1

# Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
offsets = tokenized_example["offset_mapping"][0]
if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
    # Move the token_start_index and token_end_index to the two ends of the answer.
    # Note: we could go after the last offset if the answer is the last word (edge case).
    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    end_position = token_end_index + 1
    print(start_position, end_position)
else:
    print("The answer is not in this feature.")

The answer is not in this feature.


And we can double check that it is indeed the theoretical answer:

In [44]:
# print(tokenizer.decode(tokenized_example["input_ids"][0][start_position: end_position+1]))
# print(answers["text"][0])

In [45]:
pad_on_right = tokenizer.padding_side == "right"

Now let's put everything together in one function we will apply to our training set. In the case of impossible answers (the answer is in another feature given by an example with a long context), we set the cls index for both the start and end position. We could also simply discard those examples from the training set if the flag allow_impossible_answers is False. Since the preprocessing is already complex enough as it is, we've kept is simple for this part.

In [46]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [47]:
features = prepare_train_features(dataset['train'][:5])

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the map method of our dataset object we created earlier. This will apply the function on all the elements of all the splits in dataset, so our training, validation and testing data will be preprocessed in one single command. Since our preprocessing changes the number of samples, we need to remove the old columns when applying it.

In [48]:
tokenized_datasets = dataset.map(prepare_train_features, batched=True, remove_columns=dataset["train"].column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

# Fine-tune the model

In [49]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForQuestionAnsweringSimple: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForQuestionAnsweringSimple from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForQuestionAnsweringSimple from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForQuestionAnsweringSimple were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [50]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-iuChatbot-ontologyDts-BertPretrainedTokenizerFast",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
    push_to_hub_model_id=f"{model_name}-IUChatbot-ontologyDts-BertPretrainedTokenizerFast",
)

/usr/local/lib/python3.7/dist-packages/transformers/training_args.py:926: FutureWarning: `--push_to_hub_model_id` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_model_id` instead and pass the full repo name to this argument (in this case nntadotzip/xlnet-base-cased-IUChatbot-ontologyDts-BertPretrainedTokenizerFast).
  FutureWarning,


In [51]:
from transformers import default_data_collator

data_collator = default_data_collator

In [52]:
# !curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash

In [53]:
# !sudo apt-get install git-lfs

In [54]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

Cloning https://huggingface.co/nntadotzip/xlnet-base-cased-IUChatbot-ontologyDts-BertPretrainedTokenizerFast into local empty directory.


In [55]:
trainer.train()

***** Running training *****
  Num examples = 3053
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1146


Epoch,Training Loss,Validation Loss
1,No log,0.469465
2,0.563300,0.336139
3,0.353300,0.348928


***** Running Evaluation *****
  Num examples = 347
  Batch size = 8
Saving model checkpoint to xlnet-base-cased-iuChatbot-ontologyDts-BertPretrainedTokenizerFast/checkpoint-500
Configuration saved in xlnet-base-cased-iuChatbot-ontologyDts-BertPretrainedTokenizerFast/checkpoint-500/config.json
Model weights saved in xlnet-base-cased-iuChatbot-ontologyDts-BertPretrainedTokenizerFast/checkpoint-500/pytorch_model.bin
tokenizer config file saved in xlnet-base-cased-iuChatbot-ontologyDts-BertPretrainedTokenizerFast/checkpoint-500/tokenizer_config.json
Special tokens file saved in xlnet-base-cased-iuChatbot-ontologyDts-BertPretrainedTokenizerFast/checkpoint-500/special_tokens_map.json
tokenizer config file saved in xlnet-base-cased-iuChatbot-ontologyDts-BertPretrainedTokenizerFast/tokenizer_config.json
Special tokens file saved in xlnet-base-cased-iuChatbot-ontologyDts-BertPretrainedTokenizerFast/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 347
  Batch size = 8
Sav

TrainOutput(global_step=1146, training_loss=0.4405730948190206, metrics={'train_runtime': 2198.3427, 'train_samples_per_second': 4.166, 'train_steps_per_second': 0.521, 'total_flos': 2592601319319552.0, 'train_loss': 0.4405730948190206, 'epoch': 3.0})

In [56]:
trainer.save_model("test-squad-trained")

Saving model checkpoint to test-squad-trained
Configuration saved in test-squad-trained/config.json
Model weights saved in test-squad-trained/pytorch_model.bin
tokenizer config file saved in test-squad-trained/tokenizer_config.json
Special tokens file saved in test-squad-trained/special_tokens_map.json


# Evaluation

In [57]:
import torch

for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()

odict_keys(['loss', 'start_logits', 'end_logits', 'mems'])

In [58]:
output.start_logits.shape, output.end_logits.shape

(torch.Size([8, 512]), torch.Size([8, 512]))

In [59]:
output.start_logits.argmax(dim=-1), output.end_logits.argmax(dim=-1)

(tensor([0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0'),
 tensor([0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0'))

In [60]:
n_best_size = 20

In [61]:
import numpy as np

start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": "" # We need to find a way to get back the original substring corresponding to the answer in the context
                }
            )

In [62]:
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [63]:
validation_features = dataset["test"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=dataset["test"].column_names
)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [64]:
raw_predictions = trainer.predict(validation_features)

The following columns in the test set  don't have a corresponding argument in `XLNetForQuestionAnsweringSimple.forward` and have been ignored: offset_mapping, example_id.
***** Running Prediction *****
  Num examples = 347
  Batch size = 8


In [65]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

In [66]:
max_answer_length = 30

In [67]:
start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
offset_mapping = validation_features[0]["offset_mapping"]
# The first feature comes from the first example. For the more general case, we will need to be match the example_id to
# an example index
context = dataset["test"][0]["context"]

# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
        # to part of the input_ids that are not in the context.
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        # Don't consider answers with a length that is either < 0 or > max_answer_length.
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
valid_answers

[{'score': 2.1635432, 'text': 'Ng'},
 {'score': 0.72769415, 'text': 'Ng'},
 {'score': -5.0544357, 'text': 'Ngành Công nghệ Thực phẩm (Mã ngành:'},
 {'score': -5.5480094, 'text': '). Ng'},
 {'score': -5.576663, 'text': ')'},
 {'score': -6.5005546, 'text': '6.0, TOEFL iBT 60)). Ng'},
 {'score': -6.529208, 'text': '6.0, TOEFL iBT 60))'},
 {'score': -7.022415, 'text': '. Ng'},
 {'score': -7.2277403, 'text': ', TOEFL iBT 60)). Ng'},
 {'score': -7.256394, 'text': ', TOEFL iBT 60))'},
 {'score': -7.6279206, 'text': '6'},
 {'score': -7.733798, 'text': ').'},
 {'score': -8.686343, 'text': '6.0, TOEFL iBT 60)).'},
 {'score': -9.087023, 'text': ','},
 {'score': -9.208204, 'text': '.'},
 {'score': -9.413529, 'text': ', TOEFL iBT 60)).'},
 {'score': -10.587511, 'text': ','},
 {'score': -11.322514, 'text': ', Cung cấp k'},
 {'score': -11.583621, 'text': ', Cung cấp kiến thức rộng,'},
 {'score': -11.652229, 'text': '6.0,'}]

In [68]:
dataset["test"][0]["answers"]

{'answer_start': [0],
 'text': ['Ngành Công nghệ Thực phẩm (Mã ngành: 7540101) Chương trình: Trong nước Chương trình đào tạo ngành CNTP tại Trường Đai học Quốc tế được thiết kế nhằm đào tạo các kỹ sư CNTP, có những nét chính như sau:, Cung cấp kiến thức rộng, cân bằng về các khoa học thực phẩm và kỹ thuật thực phẩm Cung cấp kiến thức cơ bản về quản lý thực phẩm (bao gồm cả quản lý sản xuất, chất lượng, vệ sinh an toàn thực phẩm và vệ sinh môi trường) Chú trọng đến kỹ năng thực hành và khả năng tiếp cận với thiết bị công nghệ cao Giảng dạy hoàn toàn bằng tiếng Anh (kỹ sư sau khi tốt nghiệp sẽ đạt trình độ tiếng Anh TOEF pBT 550 hoặc tương đương (IELTS 6.0, TOEFL iBT 60)).']}

In [69]:
import collections

examples = dataset["test"]
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

In [70]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        # if not squad_v2:
            # predictions[example["id"]] = best_answer["text"]
        # else:
        answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
        predictions[example["id"]] = answer

    return predictions

In [71]:
final_predictions = postprocess_qa_predictions(dataset["test"], validation_features, raw_predictions.predictions)

Post-processing 19 example predictions split into 347 features.


  0%|          | 0/19 [00:00<?, ?it/s]

In [72]:
from datasets import load_metric
metric = load_metric("squad_v2")

Downloading:   0%|          | 0.00/2.26k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.18k [00:00<?, ?B/s]

In [73]:
formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in dataset["test"]]
metric.compute(predictions=formatted_predictions, references=references)

{'HasAns_exact': 0.0,
 'HasAns_f1': 0.0,
 'HasAns_total': 19,
 'best_exact': 0.0,
 'best_exact_thresh': 0.0,
 'best_f1': 0.0,
 'best_f1_thresh': 0.0,
 'exact': 0.0,
 'f1': 0.0,
 'total': 19}

In [74]:
trainer.push_to_hub()

Saving model checkpoint to xlnet-base-cased-iuChatbot-ontologyDts-BertPretrainedTokenizerFast
Configuration saved in xlnet-base-cased-iuChatbot-ontologyDts-BertPretrainedTokenizerFast/config.json
Model weights saved in xlnet-base-cased-iuChatbot-ontologyDts-BertPretrainedTokenizerFast/pytorch_model.bin
tokenizer config file saved in xlnet-base-cased-iuChatbot-ontologyDts-BertPretrainedTokenizerFast/tokenizer_config.json
Special tokens file saved in xlnet-base-cased-iuChatbot-ontologyDts-BertPretrainedTokenizerFast/special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.33k/445M [00:00<?, ?B/s]

Upload file runs/Jan20_17-21-29_f229a5e268a8/events.out.tfevents.1642699304.f229a5e268a8.74.0:  65%|######4   …

To https://huggingface.co/nntadotzip/xlnet-base-cased-IUChatbot-ontologyDts-BertPretrainedTokenizerFast
   0dfe8aa..86dc596  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Question Answering', 'type': 'question-answering'}}
To https://huggingface.co/nntadotzip/xlnet-base-cased-IUChatbot-ontologyDts-BertPretrainedTokenizerFast
   86dc596..516f02e  main -> main



'https://huggingface.co/nntadotzip/xlnet-base-cased-IUChatbot-ontologyDts-BertPretrainedTokenizerFast/commit/86dc596fcab9afc6c3324fe0661f287ca0f0f652'